In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#-----------------------------------Ubicacion estaciones precipitacion----------------------------------
def abrirArchivos(ruta, letra, columnas):
    """ 
    Returns the dataframe of the data opened

            Parameters:
                    ruta (String): A string with the location of the file
                    letra (String): A string with the identification of the data type -> P=Precipitation, T=Temperature

            Returns:
                    df_datos (Dataframe): Dataframe with the information of precipitation or temperature 

    """
    if letra == "P":
        df_datos = pd.read_csv(ruta, usecols = columnas)
        #df_datos = df_datos.fillna(-1)
        #df_datos.fecha = pd.to_datetime(df_datos.fecha)

    else:
        df_datos = pd.read_csv(ruta, usecols = columnas)
        #df_datos = df_datos.fillna(-99)
        #df_datos.fecha = pd.to_datetime(df_datos.fecha, usecols = columnas)

    return df_datos


def ubicacion_estaciones(ruta_ub):
    """     
    Returns the dataframe of the location of stations

            Parameters:
                    ruta_ub (String): A string with the location of the file

            Returns:
                    df_ubic (Dataframe): Dataframe with the information of the location of the stations
     """

    df_ubic = pd.read_csv(ruta_ub, usecols=['CODIGO','altitud','longitud','latitud'])
    #df_ubic = df_ubic_precdf_ubicip_sup.round(decimals=2)

    return df_ubic

def calcEvapotranspiracion(df_tempSup, df_ub, df_radiacion,tempGet):
    """     
    Returns the dataframe of the evapotranspiration calculated with Turc Modified

            Parameters:
                    df_tempSup (Dataframe): A pandas dataframe with temperature data
                    df_ub (Dataframe): A pandas dataframe with the location of the stations
                    df_radiacion (Dataframe): A pandas dataframe with the location of the stations
                    tempGet (list): List of the temperature stations selected

            Returns:
                    df_tempSup (Dataframe): Dataframe of the evapotranspiration calculated
    """

    df_new = pd.DataFrame()
    HR = [82.065943, 82.91073, 85.125509, 85.536313, 85.341042, 83.060406, 80.410283, 78.7061, 79.407464, 83.32468, 85.336647, 84.15651]
    meses = ['Ene',	'Feb',	'Mar',	'Abr',	'May',	'Jun',	'Jul',	'Ago',	'Sep',	'Oct',	'Nov',	'Dic']
    K = [0.4, 0.37, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]

    df_tempSup['date'] = pd.date_range(start='01/01/1950', periods=len(df_tempSup), freq='D')
    df_tempSup.set_index('date', inplace=True)
    #df_tempSup = df_tempSup.groupby([(df_tempSup.index.month)]).mean()

    for i in df_tempSup.columns:
        df_new = df_new.append(df_ub.loc[df_ub.index == i])

    df_radiacion = pd.concat([df_radiacion,df_new[['latitud']]],ignore_index = False).sort_values(['latitud'], ascending=True)
    df_radiacion = df_radiacion.interpolate()

    df_evapot = pd.merge(df_new, df_radiacion, left_index=True, right_index=True)

    
    for n, j, i, in zip(range(1,13), K, HR):
        df_tempSup[df_tempSup.index.month.isin([n])] = (j * ((df_tempSup[df_tempSup.index.month.isin([n])])/((df_tempSup[df_tempSup.index.month.isin([n])])+15))) * (1+((50-i)/70))

    for m in meses:
        df_evapot[[str(m)]] = (df_evapot[[str(m)]]+50)

    for l in tempGet:
        for n, m in zip(range(1,13),meses):
            
            df_tempSup[l][df_tempSup.index.month.isin([n])] = (df_tempSup[l][df_tempSup.index.month.isin([n])] * df_evapot.loc[df_evapot.index == l, m].values[0].squeeze())

    df_tempSup = df_tempSup.fillna(-1)
    #-----------------------Modificar esta linea para generar los otros archivo de la evapotranspiracion----------
    df_tempSup = df_tempSup.groupby([(df_tempSup.index.month)]).mean()
    #-------------------------------------------------------------------------------------------------------------
    df_tempSup.reset_index(inplace=True)

    del df_tempSup['date']
    return df_tempSup

#--------------------------------Información de temperatura-------------------------------------------
filenameTemp = 'C:/Users/Bender/Desktop/Mapas_ERA/Anexos_Final/Anexo A series IDEAM bruta/temperatura_sup.csv'
cols = pd.read_csv(filenameTemp, nrows=1).columns
df_temp_sup = pd.read_csv(filenameTemp, nrows=1, usecols=cols[1:])#, usecols = columnas)
df_temp_sup = df_temp_sup.round(decimals=2)

#Estaciones para evapotranspiracion
ruta_evapot = 'C:/Users/Bender/Desktop/Mapas_ERA/Anexos_Final/Anexo A series IDEAM bruta/estaciones_sup_evapot.csv'
cols_evapot = pd.read_csv(ruta_evapot, nrows=1).columns
df_evapot_sup = pd.read_csv(ruta_evapot, usecols=cols_evapot[0:1])

estacionesTemp = df_temp_sup.columns.values
estacionesEvapot = df_evapot_sup['CODIGO'].to_numpy()

#----------------Evapotranspiracion--------------------
rutaT = 'C:/Users/Bender/Desktop/Mapas_ERA/Anexos_Final/Anexo A series IDEAM bruta/temperatura_sup.csv'
df_datos_evapot = abrirArchivos(rutaT, "T", df_temp_sup.columns)

#--------------------------------Ubicacion estaciones-----------------------------------
ruta_ub = 'C:/Users/Bender/Desktop/Mapas_ERA/Anexos_Final/Ubicacion_puntos.csv'
df_ub = ubicacion_estaciones(ruta_ub)
#print(df_ub_precipSup.head())
df_ub2 = df_ub.set_index('CODIGO')

## Tabla radiacion
ruta_rad = 'C:/Users/Bender/Desktop/Mapas_ERA/Anexos_Final/Anexo A series IDEAM bruta/radiacion.csv'
df_radiacion = pd.read_csv(ruta_rad)
df_radiacion.rename(columns={'Latitud': 'latitud'}, inplace=True)


df_evapot = calcEvapotranspiracion(df_datos_evapot, df_ub2, df_radiacion, df_temp_sup.columns)

df_evapot

,22050100,23010020,23010080,23025040,21210080,21210110,21210120,21210200,21215080,21220050,...,X11-N5.15E-75.15,X10-N5.15E-75.25,Y15-N5.25E-74.75,Y14-N5.25E-74.85,Y13-N5.25E-74.95,Y12-N5.25E-75.05,Y11-N5.25E-75.15,Y10-N5.25E-75.25,Y9-N5.25E-75.35,Z10-N5.35E-75.25
0,-1.0,29.392719,25.548591,34.626248,-1.0,-1.0,22.596617,32.531911,39.392145,32.687122,...,1.499822,2.157610,9.967804,7.519761,9.878280,11.057285,10.381379,10.350940,11.489492,6.850749
1,-1.0,30.205060,26.274914,35.710320,-1.0,-1.0,24.256062,33.330771,40.418648,33.479256,...,-3.123677,3.314128,7.204700,5.614865,7.375994,8.001825,7.242262,7.354455,8.105239,6.297472
2,-1.0,28.597897,24.819499,33.732655,-1.0,-1.0,21.794643,32.783021,35.978849,32.920802,...,1.298492,-7.735246,6.085856,3.764518,5.529988,5.737915,5.797908,5.914100,6.620790,4.039223
3,-1.0,27.681028,23.984928,32.701433,-1.0,-1.0,22.280210,29.690685,32.891803,29.800526,...,2.025247,-0.634755,4.730950,4.015572,6.133809,6.681616,7.254805,7.216916,8.176676,5.461824
4,-1.0,25.740397,22.281737,29.160659,-1.0,-1.0,18.902486,29.781880,32.207027,29.884235,...,-1.347565,-0.229052,5.219454,4.947801,7.338619,7.808464,8.164902,8.130417,8.479566,5.737406
5,-1.0,27.206892,23.592114,30.598016,-1.0,-1.0,23.147692,34.097007,35.206394,34.205883,...,-0.593974,-1.462993,6.242851,6.180172,9.392953,10.506476,10.822156,10.744913,11.399920,6.453389
6,-1.0,29.459097,25.627662,33.262768,-1.0,-1.0,24.225145,34.609986,40.506194,34.721953,...,0.424792,0.933915,7.040921,6.813624,9.152249,9.552206,10.291199,10.645324,11.138806,6.560558
7,-1.0,30.724164,26.795928,36.975568,-1.0,-1.0,26.908533,38.303257,42.762941,38.430456,...,1.463197,1.891050,-3.173524,5.621266,7.841117,8.765037,8.599824,8.160053,8.735247,5.159429
8,-1.0,32.716138,28.491039,40.748715,-1.0,-1.0,30.618167,41.547120,47.004072,41.707531,...,44.220721,2.932243,8.047187,5.729256,8.342453,9.142046,9.372790,8.941142,9.738244,7.087833
9,-1.0,29.558453,25.632410,34.944837,-1.0,-1.0,25.927079,35.565754,36.361031,35.727359,...,4.047257,3.816617,8.836800,6.444422,9.458010,9.236391,9.189650,9.324382,9.976115,7.222753


In [14]:

#----------------Evapotranspiracion--------------------
rutaT = 'C:/Users/Bender/Desktop/Mapas_ERA/Anexos_Final/Anexo A series IDEAM bruta/temperatura_sup.csv'
df_datos_evapot = abrirArchivos(rutaT, "T", df_temp_sup.columns)

df_tempSup = df_datos_evapot.copy()

df_tempSup['date'] = pd.date_range(start='01/01/1950', periods=len(df_tempSup), freq='D')
df_tempSup.set_index('date', inplace=True)
#df_tempSup = df_tempSup.groupby([(df_tempSup.index.month)]).mean()
df_tempSup

,22050100,23010020,23010080,23025040,21210080,21210110,21210120,21210200,21215080,21220050,...,X11-N5.15E-75.15,X10-N5.15E-75.25,Y15-N5.25E-74.75,Y14-N5.25E-74.85,Y13-N5.25E-74.95,Y12-N5.25E-75.05,Y11-N5.25E-75.15,Y10-N5.25E-75.25,Y9-N5.25E-75.35,Z10-N5.35E-75.25
date,,,,,,,,,,,,,,,,,,,,,
1950-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1950-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1950-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1950-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1950-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_tempSup = df_tempSup.groupby([(df_tempSup.index.month)]).mean()
df_tempSup

,22050100,23010020,23010080,23025040,21210080,21210110,21210120,21210200,21215080,21220050,...,X11-N5.15E-75.15,X10-N5.15E-75.25,Y15-N5.25E-74.75,Y14-N5.25E-74.85,Y13-N5.25E-74.95,Y12-N5.25E-75.05,Y11-N5.25E-75.15,Y10-N5.25E-75.25,Y9-N5.25E-75.35,Z10-N5.35E-75.25
date,,,,,,,,,,,,,,,,,,,,,
1,NaN,22.801197,16.642279,27.724749,NaN,NaN,20.079592,29.135235,31.121689,29.481020,...,4.841884,4.729375,13.801951,12.617770,19.944588,22.399627,23.457729,25.633220,26.599547,19.295220
2,NaN,23.032084,16.810801,27.995907,NaN,NaN,20.099696,29.229852,31.293383,29.576760,...,5.124045,5.500300,14.134667,13.196943,20.599579,23.115897,24.520057,26.398488,27.476432,20.345060
3,NaN,22.455299,16.389813,27.826725,NaN,NaN,19.830335,28.862353,30.390788,29.204900,...,4.043275,3.453605,12.638571,13.192778,20.533941,22.734821,24.265030,26.183252,27.275254,21.214231
4,NaN,22.090659,16.123667,27.629161,NaN,NaN,19.395915,28.628824,29.621400,28.968599,...,3.235136,3.598238,12.816376,12.533486,20.243369,22.790576,23.738020,25.748154,26.350324,21.062754
5,NaN,21.920493,15.999465,27.624709,NaN,NaN,19.283928,28.665104,29.710693,29.005309,...,1.501795,2.240996,11.615077,12.575822,20.441529,22.494837,24.022129,25.825768,26.547097,20.568223
6,NaN,22.373051,16.329781,27.951440,NaN,NaN,19.732334,29.400761,30.526683,29.749697,...,1.985184,1.955490,12.937385,13.662936,21.023630,23.950565,24.887273,26.521655,28.012083,20.982732
7,NaN,23.201026,16.934109,28.692708,NaN,NaN,20.552959,30.687005,31.701906,31.051207,...,2.727686,3.331913,12.574475,13.430478,20.481069,23.573502,25.314419,27.309403,28.609309,21.171667
8,NaN,23.906610,17.449105,29.366667,NaN,NaN,20.909474,31.677771,32.769040,32.053731,...,4.271783,4.390310,14.039209,13.604078,21.007383,23.460485,25.029306,26.922637,28.262810,20.577351
9,NaN,23.364597,17.053497,28.628700,NaN,NaN,20.410486,30.774524,32.104070,31.139765,...,5.605685,5.189300,13.838672,12.069485,20.818586,22.751923,24.227005,26.130876,27.354634,20.676069
